# 微调生成模型

# 微调生成模型

在本章中，我们将采用一个预训练的文本生成模型，并回顾对其进行微调的过程。这个微调步骤是产生高质量模型的关键，也是我们工具箱中使模型适应特定期望行为的重要工具。微调使我们能够使模型适应特定的数据集或领域。

在本章中，我们将在两种最常用的方法中微调文本生成模型，监督微调和偏好调优。我们将探索微调预训练文本生成模型的变革潜力，使其成为更有效的工具。

## LLM训练的三个步骤:预训练、监督微调和偏好调整

创建高质量的LLM有三个常见步骤:

1. 语言建模

创建高质量LLM的第一步是在一个或多个大规模文本数据集上进行预训练(图12-1)。在训练过程中，它尝试预测下一个词元，以准确地学习文本中发现的语言和语义表示。正如我们之前在第3章和第11章中看到的，这被称为语言建模，是一种自监督方法。

这就产生了一个基础模型，通常也被称为预训练（pretrained）模型或基础（foundation）模型。基础模型是训练过程的关键一环，但对于最终使用它的用户来说更难处理。因此，下一步——微调是很重要的环节。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4c84971c2e2d4e1bafe24e676a1a0a0a/0d363d7a-e267-4e51-bf17-37d487d7d041.png">  
</div>

2. 监督微调

如果大语言模型能很好地响应指令，并努力遵循指令，就会更有用。比如当人类要求模型写一篇文章时，他们期望模型生成文章，而不是列出其他指令(这是基础模型可能会做的事情)。

通过监督微调(SFT)，我们可以调整基本模型以遵循指令。在这个微调过程中，基本模型的参数被更新为更符合我们的目标任务，就像遵循指令一样。与预训练模型一样，它是使用next-token预测来训练的，但它不是只预测下一个词元，而是基于用户输入来预测(图12-2)。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4c84971c2e2d4e1bafe24e676a1a0a0a/fb6f620c-2509-4217-9331-5fd18c466f80.png">  
</div>

SFT也可以用于其他任务，如分类，但通常用于从基本生成模型到指令(或聊天)生成模型。

3. 偏好调优

最后一步进一步提高了模型的质量，使其更符合AI安全或人类偏好的预期行为。这被称为偏好调优。偏好调优是一种微调的形式，顾名思义，它将模型的输出与我们的偏好保持一致，这些偏好是由我们给它的数据定义的。与SFT一样，它可以改进原始模型，但在训练过程中还可以提取输出的偏好。这三个步骤如图12-3所示，并演示了从未经训练的体系结构开始到以偏好调优的LLM结束的过程。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4c84971c2e2d4e1bafe24e676a1a0a0a/de1bb36f-7b70-4052-9be2-bb33d662c00a.png">  
</div>

在本章中，我们使用一个已经在大量数据集上训练过的基本模型，并探索如何使用两种微调策略对其进行微调。对于每种方法，我们先从理论基础开始，然后再在实践中使用它们。

## 监督微调

在大型数据集上预训练模型的目的是使其能够再现语言及其含义。在这个过程中，模型学习完成输入短语，如图12-4所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/40dd35dd-390e-4778-a762-1cd10af08eac.png">  
</div>

这个例子还说明了模型并没有被训练成遵循指令，它只是试图完成一个问题，而不是回答它(图12-5)。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/5b9343e5-7156-423a-848c-a5dde9c9de12.png">  
</div>

我们可以使用这个基本模型，并通过对其进行微调，使其适应某些用例，例如遵循说明。

### 完整的微调

最常见的微调过程是全面微调。就像预训练一个大语言模型一样，这个过程包括更新一个模型的所有参数，使其与你的目标任务一致。主要的区别在于，我们现在使用的是一个更小但有标签的数据集，而预训练过程是在一个没有任何标签的大数据集上完成的(图12-6)。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/e0319025-99b6-48ae-b4db-e71130150327.png">  
</div>

我们可以使用任何标记的数据进行全面微调，这也使得它成为学习特定领域表示的一种很好的技术。为了使我们的大语言模型遵循指示，我们需要“问题-回答”数据。这个数据，如图12-7所示，是用户提出的带有相应答案的查询。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/d1fc1d26-9248-4806-b938-093c73606164.png">  
</div>

在完全微调期间，模型接受输入(指令)并使用下一个词元预测输出(响应)。反过来，它不会产生新的问题，而是会按照指令进行操作。

### 参数高效微调

更新模型的所有参数有很大的潜力可以提高其性能，但也有一些缺点。训练成本高，训练时间慢，并且需要大量的存储空间。为了解决这些问题，人们开始关注参数高效微调(PEFT)替代方案，这些替代方案侧重于以更高的计算效率对预训练模型进行微调。

#### 适配器
适配器是许多基于PEFT的技术的核心组件。该方法在Transformer内部提出了一组额外的模块化组件，可以对这些组件进行微调，以提高模型在特定任务上的性能，而无需对所有模型权重进行微调。这节省了大量的时间和计算。

“NLP的参数高效迁移学习”一文中描述了适配器，该论文表明，为任务微调BERT的3.6%的参数可以产生与微调所有模型权重相当的性能。在GLUE基准测试中，作者表明他们达到了完全微调性能的0.4%以内。在单个Transformer块中，论文提出的架构将适配器放置在注意层和前馈神经网络之后，如图12-8所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/c587a2d1-e586-44f4-b50e-b25e5e37c205.png">  
</div>

然而，仅仅改变一个Transformer块是不够的，所以这些组件是模型中每个块的一部分，如图12-9所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/299c899c-3e45-4d8f-8f27-23db2ad44641.png">  
</div>

像这样在模型中查看所有适配器的组件使我们能够看到如图12-10所示的单个适配器，它是这些组件跨越模型的所有块的集合。例如，适配器1可以是医学文本分类方面的专家，而适配器2可以专门研究命名实体识别(NER)。你可以从https://oreil.ly/XraXg 下载专门的适配器。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62d624d724844c63b6dd434d1828bf52/01cc0117-6043-41f1-a2f1-4257ef77355e.png">  
</div>

论文《AdapterHub: A framework for adaptors》介绍了AdapterHub作为共享适配器的中央存储库。这些早期的适配器中，有很多更侧重于BERT架构。最近，这个概念被应用到文本生成transformer中，如“LLaMA-Adapter: zero-init attention的语言模型的高效微调”等论文。

#### Low-Rank Adaptation (LoRA)
作为适配器的替代方案，引入了低秩自适应(low-rank adaptation, LoRA)，在撰写本文时，它是一种广泛使用且有效的PEFT技术。LoRA是一种技术(与适配器一样)，它只需要更新一小组参数。如图12-11所示，它创建一个基本模型的小子集来进行微调，而不是向模型添加层。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/7235f09c-d88d-4dc2-aea8-4549fd868bf4.png">  
</div>

像适配器一样，这个子集允许更快的微调，因为我们只需要更新基本模型的一小部分。我们通过用较小的矩阵近似原始LLM附带的大矩阵来创建这个参数子集。然后，我们可以使用这些较小的矩阵作为替代品，并对它们进行微调，而不是原始的大矩阵。以我们在图12-12中看到的10 × 10矩阵为例。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/e7ed1537-8260-4846-8e82-3d16028ea551.png">  
</div>

我们可以想出两个更小的矩阵，当它们相乘时，重建一个10 × 10的矩阵。这是一个重要的效率优势，因为我们现在只有20个权重(10 + 10)，而不是使用100个权重(10乘以10)，如图12-13所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/790f21d0-50d1-4244-83cb-7935d5fc2e1b.png">  
</div>

在训练过程中，我们只需要更新这些较小的矩阵，而不是完整的权值变化。然后更新的变化矩阵(较小的矩阵)与完整的(冻结的)权重组合，如图12-14所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/4afa9667-c28d-43df-8436-d79ad1d6342c.png">  
</div>

但你可能会怀疑性能会下降。你可能是对的。但这种权衡在哪里有意义呢?

像《内在维数解释语言模型微调的有效性》这样的论文证明了语言模型“具有非常低的内在维数”。这意味着我们可以找到甚至近似LLM的大量矩阵的小秩。例如，像GPT-3这样的175B模型，其96个Transformer块中的每个块中都有一个12288 × 12288的权重矩阵。也就是1.5亿个参数。如果我们可以成功地将这个矩阵调整到8级，那只需要两个12,288 × 2矩阵，每个块就会产生197K个参数。正如前面引用的LoRA论文中进一步解释的那样，这些都是速度、存储和计算方面的主要节省。

这种较小的表示非常灵活，因为您可以选择对基本模型的哪些部分进行微调。例如，我们只能对每个Transformer层中的Query和Value权重矩阵进行微调。

压缩模型以获得(更)高效的训练
我们可以通过在将模型的原始权重投影到更小的矩阵之前减少它们的内存需求来提高LoRA的效率。LLM的权重是具有给定精度的数值，可以用float64或float32这样的比特数来表示。如图12-15所示，如果我们降低表示一个值的位数，我们得到的结果精度就会降低。然而，如果我们降低比特数，我们也降低了该模型的内存需求。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/585d390b-47d3-4123-a768-625bfac3db85.png">  
</div>

通过量化，我们的目标是在降低比特数的同时仍然准确地表示原始权重值。然而，如图12-16所示，当直接将较高精度值映射到较低精度值时，多个较高精度值可能最终由相同的较低精度值表示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/7698402d-7bb3-4803-bb84-8f53bac4fd0d.png">  
</div>

相反，QLoRA (LoRA的量子化版本)的作者找到了一种方法，可以从较高的位数到较低的值，反之亦然，而不会与原始权重有太大的区别。

他们使用逐块量化的方法，将某些精度值较高的块映射到精度值较低的块。他们没有直接将更高的精度映射到更低的精度值，而是创建了额外的块，允许量化相似的权重。如图12-17所示，这导致了可以用较低精度精确表示的值。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/766b25a7-a49a-45e4-9636-1b28a2572b1d.png">  
</div>

神经网络的一个很好的特性是，它们的值通常在-1和1之间正态分布。这个属性允许我们根据它们的相对密度将原始权重分类到较低的位，如图12-18所示。权重之间的映射更有效，因为它考虑了权重的相对频率。这也减少了异常值的问题。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/affd9cae-f291-423c-9f7a-d59d248f7033.png">  
</div>

结合块量化，这种归一化过程允许用低精度值精确表示高精度值，而LLM的性能只会有很小的下降。因此，我们可以从16位浮点表示变为可怜的4位规范化浮点表示。4位表示法显著降低了LLM在训练过程中的内存需求。请注意，大语言模型的量化通常也有助于推理，因为量化的大语言模型尺寸更小，因此需要更少的VRAM。

还有一些更优雅的方法可以进一步优化这一点，比如双量化和分页优化器，您可以在前面讨论的QLoRA论文中了解更多。想要了解完整且高度可视化的量化指南，请参阅这篇博客文章(https://newsletter.maartengrootendorst.com/p/a-visual-guide-to-quantization)。

## 使用QLoRA进行指令调优
既然我们已经探索了QLoRA的工作原理，那么让我们将这些知识付诸实践吧!在本节中，我们将对Llama的一个完全开源的小型版本TinyLlama进行微调，以遵循使用QLoRA过程的说明。可以把这个模型看作是一个基础模型或预训练模型，它经过了语言建模的训练，但还不能遵循指令。

### 模板化指令数据
为了让LLM遵循指令，我们需要准备遵循聊天模板的指令数据。这个聊天模板，如图12-19所示，区分了LLM生成的内容和用户生成的内容。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2d9e91e2331e4025a9049c7bd171026f/1128c43d-5207-454b-ba6f-07eb48601d84.png">  
</div>

我们选择这个聊天模板在整个示例中使用，因为TinyLlama的聊天版本使用相同的格式。我们使用的数据是UltraChat数据集的一小部分。该数据集是原始UltraChat数据集的过滤版本，该数据集包含用户和LLM之间的近200k个对话。

我们创建一个函数， format_prompt，以确保对话遵循这个模板：

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load a tokenizer to use its chat template
template_tokenizer = AutoTokenizer.from_pretrained(
    "TinyLlama/TinyLlama-1.1BChat-v1.0"
)

def format_prompt(example):
    """Format the prompt to using the <|user|> template TinyLLama is using"""

    # Format answers
    chat = example["messages"]
    prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

    return {"text": prompt}

# Load and format the data using the template TinyLLama is using
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
      .shuffle(seed=42)
      .select(range(3000))
)
dataset = dataset.map(format_prompt)

我们选择3000个文档的子集来减少训练时间，但是你可以增加这个值来获得更准确的结果，使用 “文本” 列，我们可以探索这些格式化的提示：

In [ ]:
# Example of formatted prompt
print(dataset["text"][2576])

## 模型量化
现在我们有了我们的数据，我们可以开始在我们的模型中加载。这就是我们在QLoRA中应用Q的地方，即量化。我们使用 bitsandbytes 包 将预训练的模型压缩为4位表示。

在 BitsAndBytesConfig，可以定义量化方案。我们遵循原始QLoRA论文中使用的步骤，并以4位(load_in_4bit)，具有标准化浮点表示形式(bnb_4bit_quant_type)和双量化(bnb_4bit_use_double_quant):



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
)

# Load the model to train on the GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    # Leave this out for regular SFT
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

这个量化过程允许我们减小原始模型的大小，同时保留大部分原始权重的精度。加载模型现在只使用~1 GB VRAM，而没有量化时需要~4 GB的VRAM。请注意，在微调期间，将需要更多的VRAM，因此加载模型所需的~1 GB VRAM不会达到上限。

## LoRA罗拉配置
接下来，我们需要使用 peft 图书馆，表示微调过程的超参数:

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

有几个参数值得一提:

r

这是压缩矩阵的秩(回想一下图12-13)增加这个值也会增加压缩矩阵的大小，导致更少的压缩，从而提高代表性。数值通常在4到64之间。

lora_alpha

控制添加到原始权重的更改量。从本质上讲，它平衡了原始模型的知识与新任务的知识。一个经验法则是选择大小为的两倍的值 r.

target_modules

控制要瞄准哪些图层。LoRA过程可以选择忽略特定的层，比如特定的投影层。这可以加快训练速度，但会降低性能，反之亦然。

摆弄这些参数是一个很有价值的实验，可以直观地了解哪些值有效，哪些值无效。您可以在Sebastian Raschka的Ahead of AI通讯中找到关于LoRA微调的额外提示的惊人资源。

## 训练配置

最后，我们需要像在第11章中那样配置我们的训练参数:

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

有几个参数值得一提:

num_train_epochs

训练轮次总数。较高的数值往往会降低性能，所以我们通常喜欢将这个数值保持在较低的水平。

learning_rate

确定每次权重更新迭代时的步长。QLoRA的作者发现，更高的学习率对于更大的模型(>33B参数)效果更好。

lr_scheduler_type

基于余弦的调度器，动态调整学习率。它会线性增加学习率，从0开始，直到达到设定值。之后，学习率会随着余弦函数的值而衰减。

optim

原始QLoRA论文中使用的页面优化器。

优化这些参数是一项艰巨的任务，并且没有相应的指导方针。它需要通过实验来找出最适合特定数据集、模型大小和目标任务的方法。

# 训练

现在我们已经准备好了所有的模型和参数，我们可以开始微调我们的模型了。我们载入 SFTTrainer 然后简单地跑 trainer.train ():

In [ ]:
from trl import SFTTrainer

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,

    # Leave this out for regular SFT
    peft_config=peft_config,
)

# Train model
trainer.train()

# Save QLoRA weights
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

在训练过程中，每10步将根据 logging_steps 参数。如果你使用的是谷歌Colab提供的免费GPU，也就是本文写作时的特斯拉T4，那么训练时间可能会长达一个小时。可以趁此机会休息一下!

## 合并权重

在我们训练完我们的QLoRA权值之后，我们仍然需要将它们与原始权值结合起来使用它们。我们以16位重新加载模型，而不是量化的4位，来合并权重。虽然在训练期间没有更新标记器，但我们将其保存到与模型相同的文件夹中，以便于访问:

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

将适配器与基本模型合并后，我们可以将其与我们之前定义的提示模板一起使用:

In [ ]:
from transformers import pipeline

# Use our predefined prompt template
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# Run our instruction-tuned model
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

汇总输出显示，模型现在紧密遵循我们的指令，这在基础模型中是不可能的。

# 评估生成模型

评估生成模型提出了一个重大挑战。生成模型被用于许多不同的用例，这使得依赖单一指标进行判断成为一项挑战。与更专业的模型不同，生成模型解决数学问题的能力并不能保证成功解决编码问题。

同时，评估这些模型至关重要，特别是在一致性很重要的生产环境中。鉴于其概率性质，生成模型不一定产生一致的输出;需要进行鲁棒性评估。

在本节中，我们将探讨几种常见的评估方法，但我们想强调目前缺乏通用标准。没有一个度量标准对所有用例都是完美的。

## 字级指标

比较生成模型的一个常见指标类别是字级指标。这些经典技术将参考数据集与生成的词元在词元(集)级别进行比较。常见的词级指标包括perplexity、 ROUGE、BLEU、和BERTScore.

值得注意的是困惑度，它衡量语言模型预测文本的效果。给定输入文本，模型预测下一个标记的可能性有多大。对于perplexity，我们假设如果一个模型给出下一个词元的高概率，它的表现会更好。换句话说，当呈现一份写得很好的文档时，模型不应该感到“困惑”。

如图12-20所示，当输入“when a measure become a”时，会询问模型“target”作为下一个单词的可能性有多大。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/11434b6c98ae4283a55ab5ecf3e1bbf3/1cf348db-8e2a-42d6-9cd1-b8e37a349608.png">  
</div>

尽管困惑和其他词级指标是理解模型置信度的有用指标，但它们并不是一个完美的衡量标准。它们不能解释生成文本的一致性、流畅性、创造性，甚至正确性。

## 基准

在语言生成和理解任务上评估生成模型的一种常用方法是基于众所周知的和公开的基准，如MMLU、GLUE、TruthfulQA、GSM8k、和hellaswg.这些基准测试为我们提供了关于基本语言理解的信息，但也提供了复杂的分析性回答，如数学问题。

除了自然语言任务，一些模型还专门研究其他领域，比如编程。这些模型倾向于在不同的基准上进行评估，比如HumanEval,由具有挑战性的编程任务组成，供模型解决。表12-1给出了生成模型的常见公共基准的概述。


|基准|描述|资源|
|---|---|---|
|MMLU|大规模多任务语言理解(Massive Multitask Language Understanding, MMLU)基准测试在57个不同的任务上测试该模型，包括分类、问答和情感分析。|https://oreil.ly/nrG_g|
|GLUE|通用语言理解评估(GLUE)基准包括涵盖广泛难度的语言理解任务。|	https://oreil.ly/LV_fb|
|TruthfulQA|TruthfulQA测量模型生成文本的真实性。|https://oreil.ly/i2Brj|
|GSM8k|GSM8k数据集包含小学数学单词问题。它具有语言多样性，由人类问题编写者创建。|https://oreil.ly/oOBXY|
|HellaSwag|HellaSwag是一个用于评估常识推理的挑战数据集。它由模型需要回答的多项选择题组成。它可以从每个问题的四个答案选项中选择一个。|https://oreil.ly/aDvBP|
|HumanEval|HumanEval基准用于评估基于164个编程问题生成的代码。|https://oreil.ly/dlJIX|


基准测试是基本了解模型在各种任务上的表现的好方法。公共基准测试的一个缺点是，模型可能会过度拟合到这些基准测试中，以产生最佳响应。此外，这些仍然是广泛的基准，可能不会涵盖非常具体的用例。最后，另一个缺点是，一些基准测试需要运行时间长(超过几个小时)的强大gpu来计算，这使得迭代变得困难。

## 排行榜
有这么多不同的基准，很难选择哪个基准最适合你的模型。每当一个模型发布时，你经常会看到它在几个基准测试上进行评估，以展示它的全面表现。

因此，排行榜的开发包含了多个基准测试。常见的排行榜是Open LLM排行榜，在撰写本文时，它包含六个基准测试，包括HellaSwag、MMLU、TruthfulQA和GSM8k。在排行榜上排名靠前的模型，假设它们没有对数据进行过拟合，通常被认为是“最佳”模型。然而，由于这些排行榜通常包含公开可用的基准，因此排行榜存在过拟合的风险。

## 自动评价

评估生成输出的部分内容是其文本的质量。例如，即使两个模型对一个问题给出相同的正确答案，它们得出答案的方式也可能不同。这通常不仅仅是关于最终答案，还包括它的构造。同样，虽然两个总结可能相似，但其中一个可能比另一个短得多，这对于一个好的总结来说通常很重要。

为了评估生成文本的质量高于最终答案的正确性，引入了LLM-as-a-judge本质上，就是要求一个单独的LLM来判断待评价LLM的质量。这种方法的一个有趣的变体是两两比较。两个不同的大语言模型将生成一个问题的答案，第三个大语言模型将作为法官宣布哪个更好。

因此，这种方法允许对开放式问题进行自动评估。一个主要的优势是，随着大语言模型的进步，他们判断输出质量的能力也在提高。换句话说，这种评价方法是随着领域的发展而发展的。

## 人的评价

虽然基准很重要，但评估的黄金标准通常被认为是人的评估。即使LLM在广泛的基准测试中得分很好，它在特定领域的任务上仍然可能得分不高。此外，基准测试并不能完全捕捉人类的偏好，前面讨论的所有方法都只是人类偏好的代理。

基于人类的评估技术的一个很好的例子是聊天机器人竞技场。19当你进入这个排行榜时，你会看到两个(匿名的)大语言模型，你可以与之互动。你问的任何问题或提示都会被发送到两个模型，你会收到它们的输出。然后，你可以决定你喜欢哪个输出。这个过程允许社区在不知道展示了哪些模型的情况下，对他们喜欢的模型进行投票。只有在你投票之后，你才能看到哪个模型生成了哪个文本。

在撰写本文时，这种方法已经生成了超过80万+的人类投票，用于计算排行榜。这些投票被用来计算基于胜率的大语言模型的相对技能水平。例如，如果排名较低的大语言模型击败排名较高的大语言模型，其排名就会发生显著变化。在国际象棋中，这被称为Elo评级系统。

因此，这种方法使用众包投票，这有助于我们了解大语言模型的质量。然而，它仍然是各种各样的用户的汇总意见，这可能与你的用例无关。

因此，没有一种完美的评估llm的方法。所有提到的方法和基准都提供了一个重要的，尽管有限的评估视角。我们的建议是根据预期的用例来评估你的大语言模型。对于编码，HumanEval将比GSM8k更符合逻辑。

但最重要的是，我们相信你是最好的评估者。人类评估仍然是黄金标准，因为大语言模型是否适用于您的预期用例取决于您。与本章中的示例一样，我们强烈建议您也尝试这些模型，并可能自己提出一些问题。例如，本书的作者是阿拉伯语(Jay Alammar)和荷兰语(Maarten Grootendorst)，当我们接触到新的模型时，我们经常用母语提问。

关于这个话题，最后要说明的是我们非常珍视的一句话:

当一种措施成为目标时，它就不再是一种好的措施。

——Goodhart’s Law

在大语言模型的背景下，当使用特定的基准时，我们倾向于针对该基准进行优化，而不考虑后果。例如，如果我们纯粹专注于优化以生成语法正确的句子，那么模型可以学习只输出一个句子:“这是一个句子。”它在语法上是正确的，但却没有告诉你它的语言理解能力。因此，该模型可能在特定的基准上表现出色，但可能以牺牲其他有用的能力为代价。

# 首选项调整 / 对齐 / RLHF

虽然我们的模型现在可以遵循指令，但我们可以通过最后的训练阶段进一步改善它的行为，使其与我们期望它在不同场景中的行为保持一致。例如，当被问到“什么是LLM?”，我们可能更喜欢一个描述LLM内部的详细答案，而不是没有进一步解释的答案“这是一个大型语言模型”。我们如何准确地将我们(人类)对一个答案的偏好与LLM的输出结合起来?

首先，回想一下，LLM接受提示并输出如图12-21所示的生成。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/11434b6c98ae4283a55ab5ecf3e1bbf3/0e38aed4-48d1-4481-a597-b5e69805f2ab.png)

我们可以请一个人(偏好评估者)来评估该模型生成的质量。假设他们给它一个特定的分数，比如4分(见图12-22)。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/11434b6c98ae4283a55ab5ecf3e1bbf3/7d355388-1c51-469b-afc9-90ebcfc90198.png)

图12-23显示了基于该分数更新模型的偏好调优步骤:

如果得分高，则更新模型以鼓励其生成更多类似于此类型的生成。

如果分数低，则更新模型以阻止这类世代的产生。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/11434b6c98ae4283a55ab5ecf3e1bbf3/c55b1a1c-7fac-493a-80f4-92062bcde5a9.png)

一如既往，我们需要很多训练样例。那么我们能自动化偏好评估吗?是的，我们可以通过训练一个不同的模型，叫做奖励模型。

# 使用奖励模型自动化偏好评估

为了实现偏好评估的自动化，我们需要在偏好调优步骤之前进行一步，即训练一个奖励模型，如图12-24所示。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/b375d6e7-a33b-4da1-ac12-6810393a83ac.png)

图12-25显示，为了创建一个奖励模型，我们取了一个指令调优模型的副本，并对其进行了轻微的修改，使其不再生成文本，而是输出单个分数。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/80a54f2f-5ea0-4443-a0de-851c82a5159e.png)



## 奖励模型的输入和输出

我们期望这个奖励模型工作的方式是，我们给它一个提示和一个代，它输出一个单一的数字，表示响应该提示的那一代的偏好/质量。图12-26展示了生成这个单一数字的奖励模型。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/ed776e33-eb49-4bfa-8304-a2eb3307168b.png)

## 训练奖励模型

我们不能直接使用奖励模型。首先需要训练它正确地为几代人打分。所以让我们得到一个模型可以学习的偏好数据集。

奖励模型训练数据集
偏好数据集的一个常见形状是，训练样例有一个提示符，其中有一个被接受的生成和一个被拒绝的生成。(细微差别:它并不总是一个好一代与坏一代;也有可能是两代人都很好，但那一代比另一代好)。图12-27展示了一个带有两个训练样例的示例偏好训练集。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/5586161d-d2b8-4d8c-ad27-2a47e7c14938.png)

生成偏好数据的一种方法是向LLM提供一个提示，并让它生成两个不同的代。如图12-28所示，我们可以询问人类标注者他们更喜欢哪一个。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/1efb70cb-56be-4e9c-af01-79cf5540c479.png)

奖励模型训练步骤
现在我们有了偏好训练数据集，我们可以继续训练奖励模型。

一个简单的步骤是，我们使用奖励模型来:

1.对被接受的一代进行评分

2.给被拒绝的一代打分

图12-29显示了训练目标:确保被接受的一代比被拒绝的一代得分更高。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/8ff3e300-bfdb-4961-872f-d38212b3546a.png)

当我们将所有内容组合在一起时，如图12-30所示，我们得到了三个阶段的偏好调整:

1.收集偏好数据

2.训练奖励模型

3.使用奖励模型对LLM进行微调(作为偏好评估器操作)

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/33362fb3-0e68-4f74-a48d-a075789157bd.png)

奖励模型是一个很好的想法，可以进一步扩展和发展。例如，《羊驼2》训练了两个奖励模型:一个为乐于助人打分，另一个为安全打分(图12-31)。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/46c4af7f-b1b7-4205-921b-59f37d378d48.png)

用训练好的奖励模型对LLM进行微调的常用方法是近端策略优化(Proximal Policy Optimization, PPO)。PPO是一种流行的强化技术，它通过确保LLM不会过多地偏离预期奖励来优化指令调优的LLM它甚至被用于训练2022年11月发布的原始ChatGPT。

## 训练无奖励模型

PPO的一个缺点是，它是一个复杂的方法，需要训练至少两个模型，奖励模型和LLM，这可能比必要的成本更高。

直接偏好优化(DPO)是PPO的一种替代方法，它消除了基于强化的学习过程我们没有使用奖励模型来判断一代的质量，而是让LLM自己来做。如图12-32所示，我们使用LLM的一个副本作为参考模型，来判断参考模型和可训练模型在可接受代和被拒绝代的质量上的变化。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/d736ea1b-ed32-49b8-a61f-947055a94d63.png)

通过计算训练过程中的这种变化，我们可以通过跟踪参考模型和可训练模型的差异来优化被接受代相对于被拒绝代的可能性。

为了计算这种移位及其相关分数，从两个模型中提取被拒绝代和被接受代的对数概率。如图12-33所示，这个过程是在一个令牌级别执行的，其中概率被组合起来计算参考模型和可训练模型之间的偏移。

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/d8e457b734b84783add5d3ab0c52f5a3/c250a1a8-10c6-4a0a-80ac-c8e28bfc02ae.png)

使用这些分数，我们可以优化可训练模型的参数，使其更有信心生成可接受的代，而不太有信心生成被拒绝的代。与PPO相比，作者发现DPO在训练过程中更稳定，更准确。由于它的稳定性，我们将使用它作为我们的首选模型来调整我们之前的指令调优模型。

# 使用DPO进行偏好调优
当我们使用拥抱脸堆栈时，偏好调优与我们之前介绍的指令调优惊人地相似，只是有一些细微的区别。我们仍将使用TinyLlama，但这一次是一个指令调整版本，首先使用完整的微调进行训练，然后进一步与DPO对齐。与我们最初的指令调优模型相比，这个LLM是在更大的数据集上训练的。

在本节中，我们将演示如何使用DPO和基于奖励的数据集进一步调整该模型。

## 模板对齐数据
我们将使用一个数据集，其中每个提示包含一个可接受的生成和一个被拒绝的生成。这个数据集部分是由ChatGPT生成的，并给出了哪些输出应该被接受，哪些被拒绝的分数:

In [ ]:
from datasets import load_dataset

def format_prompt(example):
    """Format the prompt to using the <|user|> template TinyLLama is using"""

    # Format answers
    system = "<|system|>\n" + example["system"] + "</s>\n"
    prompt = "<|user|>\n" + example["input"] + "</s>\n<|assistant|>\n"
    chosen = example["chosen"] + "</s>\n"
    rejected = example["rejected"] + "</s>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Apply formatting to the dataset and select relatively short answers
dpo_dataset = load_dataset(
    "argilla/distilabel-intel-orca-dpo-pairs", split="train"
)
dpo_dataset = dpo_dataset.filter(
    lambda r: 
        r["status"] != "tie" and 
        r["chosen_score"] >= 8 and 
        not r["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(
    format_prompt,  remove_columns=dpo_dataset.column_names
)
dpo_dataset

注意，我们应用了额外的过滤，进一步将数据的大小从最初的13000个样本减少到大约6000个样本。

## 模型量化
我们加载我们的基本模型，并用我们之前创建的LoRA加载它。和之前一样，我们量化模型来减少训练所需的VRAM:

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig, AutoTokenizer

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
)

# Merge LoRA and base model
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
    quantization_config=bnb_config,
)
merged_model = model.merge_and_unload()

# Load LLaMA tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

接下来，我们使用与之前相同的LoRA配置来执行DPO训练:

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # Layers to target
     ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj", "down_proj"]
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

## 训练配置
为了简单起见，我们将使用与之前相同的训练参数，但有一点不同。为了说明目的，我们没有运行单个epoch(可能需要长达两个小时)，而是运行200步。此外，我们还添加了 warmup_ratio 参数，将学习率从0提高到 learning_rate 我们为前10%的步骤设置的值。通过在开始时保持一个小的学习率(即预热期)，我们允许模型在应用更大的学习率之前对数据进行调整，从而避免有害的分歧:

In [ ]:
from trl import DPOConfig

output_dir = "./results"

# Training arguments
training_arguments = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1
)

## 训练
现在我们已经准备好了所有的模型和参数，我们可以开始微调我们的模型了:

In [ ]:
from trl import DPOTrainer

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model,
    args=training_arguments,
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=512,
)

# Fine-tune model with DPO
dpo_trainer.train()

# Save adapter
dpo_trainer.model.save_pretrained("TinyLlama-1.1B-dpo-qlora")

我们已经创建了第二个适配器。为了合并两个适配器，我们迭代地将适配器与基本模型合并:

In [ ]:
from peft import PeftModel

# Merge LoRA and base model
model = AutoPeftModelForCausalLM.from_pretrained(
    "TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
)
sft_model = model.merge_and_unload()

# Merge DPO LoRA and SFT model
dpo_model = PeftModel.from_pretrained(
    sft_model,
    "TinyLlama-1.1B-dpo-qlora",
    device_map="auto",
)
dpo_model = dpo_model.merge_and_unload()

这种SFT+DPO的组合是一个很好的方法，首先微调你的模型来执行基本的聊天，然后让它的答案与人类的偏好保持一致。然而，这确实是有代价的，因为我们需要执行两个训练循环，并可能在两个过程中调整参数。

自从DPO发布以来，已经开发了对齐首选项的新方法。值得注意的是优势比偏好优化(ORPO)，这是一个将SFT和DPO结合成一个单一训练过程的过程它消除了执行两个单独的训练循环的需要，进一步简化了训练过程，同时允许使用QLoRA。

# 总结
在本章中，我们探讨了微调预训练llm的不同步骤。我们通过低秩自适应(LoRA)技术利用参数有效微调(PEFT)进行微调。我们解释了如何通过量化来扩展LoRA，量化是一种在表示模型和适配器参数时减少内存约束的技术。

我们探索的微调过程有两个步骤。第一步，我们在预训练的LLM上使用指令数据进行监督微调，通常称为指令调优。这导致了一个具有类似聊天行为的模型，并且可以紧密地遵循指令。

在第二步中，我们通过对校准数据进行微调来进一步改进模型，校准数据代表了哪种类型的答案比其他类型更受欢迎。这个过程被称为偏好调优，将人类的偏好提炼到之前的指令调优模型中。

总的来说，本章展示了对预训练的LLM进行微调的两个主要步骤，以及这如何导致更准确和信息丰富的输出。